# SuperAI Season 4 - Level 2 Hackathon - Table Question Answering

In [ ]:
"""Path to Langchain"""

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

In [ ]:
import os
import pandas as pd

In [88]:
load_dotenv(dotenv_path='.notebooks/.env')
# Now you can access the environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [89]:
llm = ChatOpenAI(model="gpt-4o" , temperature = 0.7)

In [ ]:
df = pd.read_csv("./datasets/TBL4-Online-Shopping-Dataset.csv")

tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df['Avg_Price'].mean()")

In [91]:
llm_with_tools = llm.bind_tools([tool], tool_choice = tool.name)

In [ ]:
system = f"""You have access to a pandas dataframe `df`. \
Here is the output of `df.head().to_markdown()`:

\`\`\`
{df.head().to_markdown()}
\`\`\`

Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

In [93]:
parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)

In [94]:
queries = pd.read_csv('./datasets/data.csv' , index_col = 'id')

In [95]:
submission = pd.read_csv('./datasets/sample_submission.csv' , index_col = 'id')

In [ ]:
chain = prompt | llm_with_tools | parser | tool

chain.invoke({"question": "What is the total number of customers?"})

In [ ]:
results = []

for idx , query in enumerate(queries['query_str']) :
    
    print(query , '\n' , results[idx])
    results.append(chain.invoke(query))

In [98]:
submission['response'] = results

In [99]:
submission.to_csv('./datasets/submission.csv')